# Setup

In [10]:
import numpy as np

with open('input.txt') as f:
    lines = f.readlines()
    
constraints = [[int(i) for i in constraint.strip().split('|')] for constraint in lines[:lines.index('\n')]]
print_order = [[int(i) for i in constraint.strip().split(',')] for constraint in  lines[lines.index('\n') + 1:]]

# Brute force approach
def does_order_satisfy_constraint(order: list) -> bool:
    for constraint in constraints:
        if constraint[0] in order and constraint[1] in order:
            if order.index(constraint[0]) > order.index(constraint[1]):
                return False
    return True

# Part 1

In [11]:
sum = 0

for order in print_order:
    if does_order_satisfy_constraint(order):
        sum += order[len(order) // 2]
        
print(sum)

7074


# Part 2

In [ ]:
# Build the graph describing the constraints
from typing import Iterable

def construct_correct_order_from_constraints(constraints: list[list[int]], verbose = True):
    constraints = constraints.copy()
    unique_pages = set()
    for constraint in constraints:
        unique_pages.add(constraint[0])
        unique_pages.add(constraint[1])

    absolute_order = [constraints[0][0], constraints[0][1]]
    del constraints[0]

    def can_number_be_placed_here(lower: int, higher: int, relevant_constraints: Iterable[list], verbose = True) -> bool:
        relevant_constraints = list(relevant_constraints)
        verbose and print(f"Trying to find out if {lower} can be placed below {higher}. Relevant constraints: {relevant_constraints}")
        for constraint in relevant_constraints:
            lower_constraint, upper_constraint = constraint
            if lower_constraint == higher and upper_constraint == lower:
                del constraints[constraints.index(constraint)]
                return False
        return True
        
    def get_relevant_constraints(one = None, two = None) -> filter:
        if one is None and two is None:
            return filter(lambda x: x[0] in absolute_order or x[1] in absolute_order, constraints)
        return filter(lambda x: (x[0] == one or x[0] == two) and (x[1] == two or x[1] == one), constraints)

    def insert_above(lower: int, upper: int, verbose = True):
        try:
            next_one_up = absolute_order[absolute_order.index(lower) + 1]
        except IndexError:
            verbose and print(f"{lower} is the last value in the list. Adding {upper} to the end.")
            absolute_order.append(upper)
            return
        relevant_constrains = get_relevant_constraints(upper, next_one_up)
        if can_number_be_placed_here(upper, next_one_up, relevant_constrains, verbose=verbose):
            verbose and print(f"{upper} is smaller than {next_one_up}. Adding {upper} above {lower}.")
            absolute_order.insert(absolute_order.index(next_one_up), upper)
            return
        else:
            verbose and print(f"{upper} is larger than {next_one_up}. Trying to insert it above {next_one_up}.")
        insert_above(next_one_up, upper, verbose=verbose)
        
    def insert_below(lower: int, upper: int, verbose = True):
        current_index = absolute_order.index(upper)
        if current_index == 0:
            verbose and print(f"{upper} is the first value in the list. Adding {lower} to the beginning.")
            absolute_order.insert(0, lower)
            return
        next_one_down = absolute_order[current_index - 1]
        relevant_constrains = get_relevant_constraints(next_one_down, lower)
        if can_number_be_placed_here(next_one_down, lower, relevant_constrains, verbose=verbose):
            verbose and print(f"{lower} is larger than {next_one_down}. Adding {lower} below {upper}.")
            absolute_order.insert(absolute_order.index(upper), lower)
            return
        else:
            verbose and print(f"{lower} is smaller than {next_one_down}. Trying to insert it below {next_one_down}.")
        insert_below(lower, next_one_down, verbose=verbose)

    while len(absolute_order) < len(unique_pages):
        relevant_constrains = list(get_relevant_constraints())
        for constraint in relevant_constrains:
            lower, upper = constraint
            
            if lower in absolute_order and upper not in absolute_order:
                verbose and print(f"Absolute order: {absolute_order}; Lower: {lower}; Upper: {upper}; relevant constraints: {constraints}")
                insert_above(lower, upper, verbose)
            if upper in absolute_order and lower not in absolute_order:
                verbose and print(f"Absolute order: {absolute_order}; Lower: {lower}; Upper: {upper}; relevant constraints: {constraints}")
                insert_below(lower, upper, verbose)
                
    # Check remaining constraints
    not_satisfied_constrains = []
    while len(constraints) > 0:
        lower, upper = constraints[0]
        if lower in absolute_order and upper in absolute_order:
            if absolute_order.index(lower) > absolute_order.index(upper):
                not_satisfied_constrains.append(constraints[0])
            del constraints[0]
    if len(not_satisfied_constrains) == 0:
        verbose and print("All constraints satisfied.")
    else:
        verbose and print("Constraints not satisfied:")
        for constraint in not_satisfied_constrains:
            verbose and print(constraint)
            return None
    verbose and print(absolute_order)
    return absolute_order

sum = 0
for order in print_order:
    if not does_order_satisfy_constraint(order):
        relevant_constraints = [constraint for constraint in constraints if constraint[0] in order and constraint[1] in order]
        sorted_order = construct_correct_order_from_constraints(relevant_constraints, verbose=False)
        sum += sorted_order[len(sorted_order) // 2]
        
print(sum)

4828
